<img src="./pictures/logo-insa.png" style="float:right; max-width: 60px; display: inline" alt="INSA" /></a>

# Battery and ESC selection
*Written by Marc Budinger, INSA Toulouse, France*

## Design graph 

The following diagram represents the design graph of the battery and ESC selection.


![DesignGraph](pictures/ESCBatteryDesignGraph.png)

> **Questions:**
* Recall the main equations to take into account during ESC and battery selection
* Orientate the arrows
* Give equations order, inputs/outputs at each step of this part of sizing procedure



### Sizing code and optimization

> Exercice: propose a sizing code for the selection of a battery and ESC.


In [1]:
from utils.model_standard import CoreModel
from utils.model_serializer import ModelSerializer

In [2]:
import math
class BatteryModel(CoreModel):
    """
    Battery model class.
    ----------
    """

    def __init__(self, **kwargs):
        super(BatteryModel, self).__init__(**kwargs)
        self.initialization()
        self.execute()
        self._update()

    def initialization(self):
        
        # Input variables
        inputs = {'V_bat_est': 1.0, 'k_Mb': 1.0, 'M_load': 1.0, 'P_el_hover': 1.0, 'P_el_takeoff': 1.0, 'Umot_takeoff': 1.0, 'Npro': 1.0}
        self.set_inputs(inputs)
        
        # Input parameters
        
        # Ref : MK-quadro
        Mbat_ref=.329 # [kg] mass
        Ebat_ref=220*3600*.329 # [J]
        
        # Ref : Turnigy K_Force 70HV 
        Pesc_ref=3108 # [W] Power
        Mesc_ref=.115 # [kg] Mass
        
        inputs = {'Mbat_ref': Mbat_ref, 'Ebat_ref': Ebat_ref, 'Pesc_ref': Pesc_ref, 'Mesc_ref': Mesc_ref}
        self.set_inputs(inputs)
        
        # Declare outputs
        outputs = ['Ncel', 'V_bat', 'Mbat', 'Ebat', 'C_bat', 'I_bat', 't_hf', 'P_esc', 'Mesc', 'Vesc']   
        self.declare_outputs(outputs)

    def execute(self):
        
        # Get input values
        V_bat_est, k_Mb, M_load, P_el_hover, P_el_takeoff, Umot_takeoff, Npro = self.get_values(['V_bat_est', 'k_Mb', 'M_load', 'P_el_hover', 'P_el_takeoff', 'Umot_takeoff', 'Npro'])
        Mbat_ref, Ebat_ref, Pesc_ref, Mesc_ref = self.get_values(['Mbat_ref', 'Ebat_ref', 'Pesc_ref', 'Mesc_ref'])
        
        #####
        
        ## Caractéristiques Batterie
        Mbat=k_Mb*M_load; #On estime la masse de la batterie par rapport à la masse de chargement utile recherchée, 
        #les 2 valeurs sont en effet corrélées. Le facteur de dimensionnement k_Mb est ensuite adapté par l'optimisation
        Ebat=Ebat_ref*(Mbat/Mbat_ref); # On calcule l'énergie embarquée dans la batterie par loi d'échelle. La relation entre
        # masse et énergie est en effet linéaire, c'est la densité massique d'énergie de la batterie. Cette valeur influencera directement 
        # la durée du vol
        # On dimensionne ensuite les caractéristiques de la batterie en fonction de la puissance requise au décollage
        # On commence par l'intensité, et cela impacte directement la structure de la batterie, le nombre de "packs"
        # (nombre de cellules en série et en parallèle pour fournir l'intensité et la tension nécessaires)
        I_bat=(P_el_takeoff*Npro)/(0.95*V_bat_est);
        Ncel=int(V_bat_est/3.7)+1;
        V_bat=Ncel*3.7;
        C_bat=Ebat/3600/V_bat; #on veut la capacité en A*h
        # En fonction de ces caractéristiques, on peut calculer l'autonomie de la batterie, le facteur 0.8 exprime le fait qu'on ne
        # peut décharger au maximum que 80% de l'énergie de la batterie pour éviter de l'endommager
        t_hf=0.8*(C_bat/I_bat)*60;
        
        ## Caractéristiques Electronique de puissance
        # La puissance maximale traitée par les élements d'électronique de puissance doit être au moins égale
        # à la puissance nécessaire au décollage (qui est le pic de puissance requise pendant un vol)
        P_esc=P_el_takeoff;
        Mesc=Mesc_ref*(P_esc/Pesc_ref); # à partir des valeurs de puissance, on calcule directement les valeurs de masse
        # par loi d'échelle, la relation est linéaire (c'est la densité massique de puissance)
        Vesc=1.84*P_esc**0.36;
        
        #####
    
        outputs = {'Ncel': Ncel, 'V_bat': V_bat, 'Mbat': Mbat, 'Ebat': Ebat, 'C_bat': C_bat, 'I_bat': I_bat,'t_hf': t_hf, 'P_esc': P_esc, 'Mesc': Mesc, 'Vesc': Vesc}
        self.set_outputs(outputs)
                
    def __str__(self):
        
        s =(("* Battery informations: \n") +
            ("** Global: \n") + 
            ("    Battery capacity = %.2f A.h" %(self.get_values(['C_bat'])/3600) + "\n") +
            ("    Battery voltage = %.2f V" %(self.get_values(['V_bat'])) + "\n") +
            ("    Battery voltage estimated= %.2f V" %(self.get_values(['V_bat_est'])) + "\n") +
            ("    Battery current (hover) estimated= %.2f A" %(self.get_values(['I_bat'])) + "\n") +
            ("* ESC informations: \n") +
            ("** Global: \n") +
            ("    ESC power estimated= %.2f W" %(self.get_values(['P_esc'])) + "\n") + 
            ("    ESC mass estimated= %.2f kg" %(self.get_values(['Mesc'])) + "\n") + 
            ("    ESC voltage estimated= %.2f V" %(self.get_values(['Vesc'])) + "\n")
           )
        return s
        

In [3]:
battery_model = BatteryModel()

print(battery_model)

ms = ModelSerializer()
path = './models_student/'
file_name = 'battery_model'
ms.save_model(battery_model, path + file_name)

* Battery informations: 
** Global: 
    Battery capacity = 0.02 A.h
    Battery voltage = 3.70 V
    Battery voltage estimated= 1.00 V
    Battery current (hover) estimated= 1.05 A
* ESC informations: 
** Global: 
    ESC power estimated= 1.00 W
    ESC mass estimated= 0.00 kg
    ESC voltage estimated= 1.84 V



In [4]:
print(battery_model)

inputs = {'Pel_takeoff': 25.0}

battery_model.evaluate(inputs, [])

print(battery_model)

* Battery informations: 
** Global: 
    Battery capacity = 0.02 A.h
    Battery voltage = 3.70 V
    Battery voltage estimated= 1.00 V
    Battery current (hover) estimated= 1.05 A
* ESC informations: 
** Global: 
    ESC power estimated= 1.00 W
    ESC mass estimated= 0.00 kg
    ESC voltage estimated= 1.84 V

* Battery informations: 
** Global: 
    Battery capacity = 0.02 A.h
    Battery voltage = 3.70 V
    Battery voltage estimated= 1.00 V
    Battery current (hover) estimated= 1.05 A
* ESC informations: 
** Global: 
    ESC power estimated= 1.00 W
    ESC mass estimated= 0.00 kg
    ESC voltage estimated= 1.84 V



### Commentaire sur l'ESC : 
- fonction : convertisseur DC/AC avec un onduleur et de la modulation de largeur d'impulsion
- risque d'être trop petit pour avoir une tension suffisante ; le courant est limité par les pertes et donc la surchauffe
- il faut une capacité de puissance satisfaisante et une constante de temps correspondante
- le take-off est la phase dimensionnante à cause des considérations thermiques

### Commentaires sur la batterie : 
- les limitations peuvent être apportées par la densité d'énergie massique et la puissance massique 
- il faut choisir une batterie typée puissance (cas take-off dimensionnant) ou énergie (cas vol horizontal dimensionnant)